In [ ]:
# Reload when code changed:
%load_ext autoreload
%autoreload 2
%pwd
import os 
import sys
path = "../"
sys.path.append(path)
#os.path.abspath("../")
print(os.path.abspath(path))

In [ ]:
import pandas as pd
import numpy as np
import json
import timeit
import time
import core
import importlib
importlib.reload(core)
import logging
importlib.reload(core) 
try:
    logging.shutdown()
    importlib.reload(logging)
except:
    pass
from event_handler import EventHandler
print(core.__file__)
pd.__version__

### Load directories

In [ ]:
root_directory = 'D:/github/ekostat_calculator'#"../" #os.getcwd()
workspace_directory = root_directory + '/workspaces' 
resource_directory = root_directory + '/resources'
#alias = 'lena'
user_id = 'test_user' #kanske ska vara off_line user?
workspace_alias = 'lena_indicator'

## Initiate EventHandler

In [ ]:
print(root_directory)
paths = {'user_id': user_id, 
         'workspace_directory': root_directory + '/workspaces', 
         'resource_directory': root_directory + '/resources', 
         'log_directory': 'D:/github' + '/log', 
         'test_data_directory': 'D:/github' + '/test_data'}

In [ ]:
t0 = time.time()
ekos = EventHandler(**paths)
#request = ekos.test_requests['request_workspace_list']
#response = ekos.request_workspace_list(request) 
#ekos.write_test_response('request_workspace_list', response)
print('-'*50)
print('Time for request: {}'.format(time.time()-t0))
# OLD: ekos = EventHandler(root_directory)

### Load existing workspace

In [ ]:
#ekos.copy_workspace(source_uuid='default_workspace', target_alias='lena_1')

In [ ]:
ekos.print_workspaces()

In [ ]:
workspace_uuid = ekos.get_unique_id_for_alias(workspace_alias = 'lena_indicator')
print(workspace_uuid)

In [ ]:
workspace_alias = ekos.get_alias_for_unique_id(workspace_unique_id = workspace_uuid)

In [ ]:
ekos.load_workspace(unique_id = workspace_uuid)
# Här får jag ofta felmeddelande:
# AttributeError: module 'core' has no attribute 'ParameterMapping'

In [ ]:
#ekos.import_default_data(workspace_alias = workspace_alias)

### Load all data in workspace

In [ ]:
#ekos.get_workspace(unique_id = workspace_uuid, alias = workspace_alias).delete_alldata_export()

In [ ]:
#%%timeit
ekos.load_data(unique_id = workspace_uuid)

In [ ]:
w = ekos.get_workspace(unique_id = workspace_uuid)
len(w.data_handler.get_all_column_data_df())

In [ ]:
print('subsetlist', w.get_subset_list())

# Step 0 

In [ ]:
w.data_handler.all_data.head()

### Apply first data filter 

In [ ]:
w.apply_data_filter(step = 0) # This sets the first level of data filter in the IndexHandler 

# Step 1 
### Set subset filter

In [ ]:
#w.copy_subset(source_uuid='default_subset', target_alias='period_2007-2012_refvalues_2013')

In [ ]:
subset_alias = 'period_2007-2012_refvalues_2006'
subset_uuid = ekos.get_unique_id_for_alias(workspace_alias = workspace_alias, subset_alias = subset_alias)
#w.set_data_filter(subset = subset_uuid, step=1, 
#                         filter_type='include_list', 
#                         filter_name='MYEAR', 
#                         data=['2007', '2008', '2009', '2010', '2011', '2012']) 


#f1 = w.get_data_filter_object(subset = subset_uuid, step=1) 
#print(f1.include_list_filter)

In [ ]:
print('subset_alias:', subset_alias, '\nsubset uuid:', subset_uuid)

In [ ]:
f1 = w.get_data_filter_object(subset = subset_uuid, step=1) 
print(f1.include_list_filter)

### Apply step 1 datafilter to subset

In [ ]:
w.apply_data_filter(subset = subset_uuid, step = 1)

# Step 2
### Load indicator settings filter

In [ ]:
w.get_step_object(step = 2, subset = subset_uuid).load_indicator_settings_filters()

### set available indicators

In [ ]:
w.get_available_indicators(subset= subset_uuid, step=2)

### Apply indicator data filter

In [ ]:
#list(zip(typeA_list, df_step1.WATER_TYPE_AREA.unique()))
#indicator_list = w.get_available_indicators(subset= subset_uuid, step=2)
#indicator_list = ['din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter','bqi','din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter', 'biov', 'chl', 'secchi']
indicator_list = ['din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter']
#indicator_list = ['biov', 'chl']
#indicator_list = ['bqi', 'secchi']

In [ ]:
for indicator in indicator_list:
    w.apply_indicator_data_filter(step = 2, 
                          subset = subset_uuid, 
                          indicator = indicator)#,
                         # water_body_list = test_wb)
    #print(w.mapping_objects['water_body'][wb])
    #print('*************************************')

# Step 3 
### Set up indicator objects

In [ ]:
w.get_step_object(step = 3, subset = subset_uuid).indicator_setup(subset_unique_id = subset_uuid) 

In [ ]:
w.get_step_object(step = 3, subset = subset_uuid).indicator_objects['din_winter']._set_water_body_indicator_df(water_body = None)

## TEST FOR WINTER YEAR

In [ ]:
def set_water_body_indicator_df(water_body, indicator_parameter):
        """
        Created:        20180215     by Lena
        Last modified:  20180328     by Lena
        df should contain:
            - all needed columns from get_filtered_data
            - referencevalues
            - maybe other info needed for indicator functions
        skapa df utifrån:
        self.index_handler
        self.tolerance_settings
        self.indicator_ref_settings
        """
        #print(water_body)
        df = w.get_filtered_data(subset = subset_uuid, step = 'step_2', water_body = water_body, indicator = indicator).copy(deep = True)                
        if len(df.VISS_EU_CD.unique()) > 1 or df.VISS_EU_CD.unique() != water_body:
            warnings.warn(message = 'Warning: get_filtered_data() returns {} waterbody. Input waterbody is {}'.format(df.VISS_EU_CD.unique(), water_body))
            #raise Exception('Error: get_filtered_data() returns {} waterbody. Input waterbody is {}'.format(df.VISS_EU_CD.unique(), water_body))
            df = df.iloc[0:0]
        df.dropna(subset = [indicator_parameter], inplace = True)
        return df

In [ ]:
#df['WATER_TYPE_AREA'].str.contains('Botten'))
gullmarn = 'SE581700-113000'
Yxlaomradet = 'SE593750-184900'
df = set_water_body_indicator_df(Yxlaomradet, 'DIN')
df.head()

In [ ]:
len(df)

In [ ]:
# add column for winter_YEAR
def winter_year(df):
    df['winter_YEAR'] = df.apply(lambda row: row.YEAR+1 if (row.MONTH in [1,2,3]) else row.YEAR, axis=1)        

In [ ]:
#winter_year(df)
#column_list = ['DIN','YEAR', 'winter_YEAR', 'MONTH', 'SDATE', 'DEPH']
#df[column_list]

In [ ]:
w.get_step_object(step = 2, subset = subset_uuid)#.get_indicator_data_filter_settings('din_winter').get_value(variable = 'MONTH_LIST', water_body = gullmarn)


In [ ]:
ind_obj = w.get_step_object(step = 3, subset = subset_uuid).indicator_objects['din_winter']

In [ ]:
ind_obj.parent_workspace_object.get_step_object(step = 2, subset = ind_obj.subset).get_indicator_data_filter_settings('din_winter').get_value(variable = 'MONTH_LIST', water_body = gullmarn)

### tests

In [ ]:
#w.get_filtered_data(subset = subset_uuid, step = 'step_2').dropna(subset = ['DIN']).VISS_EU_CD.unique()

### CALCULATE STATUS

In [ ]:
w.get_step_object(step = 3, subset = subset_uuid).calculate_status(indicator_list = ['din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter'])
#'din_winter','ntot_summer', 'ntot_winter', 'dip_winter', 'ptot_summer', 'ptot_winter'

### CALCULATE QUALITY ELEMENTS

In [ ]:
w.get_step_object(step = 3, subset = subset_uuid).calculate_quality_element(subset_unique_id = subset_uuid, quality_element = 'Nutrients', class_name = 'QualityElementNutrients')

In [ ]:
#w.get_step_object(step = 3, subset = subset_uuid).quality_element['Nutrients'].results[['VISS_EU_CD','WATER_BODY_NAME','WATER_TYPE_AREA','STATUS_NUTRIENTS','mean_EQR','MEAN_N_EQR','EQR_N_winter_mean','global_EQR_ntot_summer','MEAN_P_EQR','EQR_P_winter_mean','global_EQR_ptot_summer'
#]].to_csv('D:/Nutrients.txt', float_format='%.3f', header = True, index = None, sep = '\t')

## Plotting results

In [ ]:
#%matplotlib inline
#import seaborn as sns
#for name, group in ind_obj.classification_results['status_by_date'].groupby('VISS_EU_CD'):
    #group['date'] = pd.to_datetime(group.SDATE)
    #group.dropna(subset = ['date', 'DIN'])
    #sns.tsplot(data = group.to_dict(), time = 'SDATE', value = 'DIN', condition = 'STATUS', legend = True)
#    group.plot('SDATE', ['DIN', 'REFERENCE_VALUE'], title = name + group.WATER_TYPE_AREA.values[0], marker ='*')

In [ ]:
#name + group.WATER_TYPE_AREA.values[0]